In [2]:
%load_ext tensorboard
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten, Dense,Layer,Dropout
from keras.callbacks import TensorBoard

In [3]:
def create_model():
  return tf.keras.models.Sequential([
    Flatten(input_shape=(28,28,3), name='layers_flatten'),
    Dense(512, activation='relu', name='layers_dense'),
    Dense(10, activation='softmax', name='layers_dense_2')
  ])

In [4]:
model = create_model()
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [5]:
epochs = 10
batch_size = 32
img_height = 28
img_width = 28

In [6]:
data_dir = "wheat_leaf"
from keras.utils import image_dataset_from_directory

dataset = image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode= 'int',
    class_names=None,
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(img_height, img_width),
    shuffle=True,
    interpolation='bilinear',
)

Found 407 files belonging to 3 classes.


In [7]:
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
num_classes = dataset.reduce(0, lambda x, _: x + 1).numpy()

In [8]:
tensorboard_callback = TensorBoard(log_dir="./log", histogram_freq=1)
def log_images(epoch, logs):
    dataset_iter = iter(dataset)
    images, _ = next(dataset_iter)
    with tf.summary.create_file_writer("./log").as_default():
        tf.summary.image("Training data", images, step=epoch)


image_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_images)

model.fit(dataset, epochs=10, batch_size=32, callbacks=[tensorboard_callback,image_callback])

Epoch 1/10
13/13 [==============================] - 22s 982ms/step - loss: 412.6892 - accuracy: 0.3612
Epoch 2/10
13/13 [==============================] - 22s 1s/step - loss: 114.2732 - accuracy: 0.5455
Epoch 3/10
13/13 [==============================] - 21s 1s/step - loss: 80.7258 - accuracy: 0.5037
Epoch 4/10
13/13 [==============================] - 21s 1s/step - loss: 69.2688 - accuracy: 0.6388
Epoch 5/10
13/13 [==============================] - 21s 1s/step - loss: 22.0339 - accuracy: 0.7273
Epoch 6/10
13/13 [==============================] - 19s 889ms/step - loss: 8.5501 - accuracy: 0.8280
Epoch 7/10
13/13 [==============================] - 12s 596ms/step - loss: 7.3179 - accuracy: 0.8501
Epoch 8/10
13/13 [==============================] - 12s 604ms/step - loss: 18.1018 - accuracy: 0.7420
Epoch 9/10
13/13 [==============================] - 13s 701ms/step - loss: 10.3437 - accuracy: 0.8182
Epoch 10/10
13/13 [==============================] - 12s 590ms/step - loss: 56.7027 - accuracy

In [9]:
%tensorboard --logdir =log

Reusing TensorBoard on port 6006 (pid 26908), started 2 days, 23:13:25 ago. (Use '!kill 26908' to kill it.)